BIOF309 Final Project

In [ ]:
##I examined the Pew Global Attitudes Survey Results, initially looking for availability of data on health-related topics
##Not every country is represented in every year, and questions asked - and which countries are asked those questions from the survey - vary frequently
##Based upon an availability of data across years and a similarity of datasets, I chose to examine Kenya, Nigeria, and South African survey results across the 2015 and 2016 surveys, focusing on trends in opinions in health-related topics and their correlation with other potentially-relevant questions
##Given the limited n of samples, I will be looking for any trends, rather than statistically significant correlations

In [ ]:
#Importing packages I plan on using
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Building PewResults DataFrame

##Countries used
countries = ["Kenya", "Nigeria", "South Africa"]

##Questions on health care as a major national problem
hcproblem2016 = [86, 93, 82] #Percentage of respondents indicating that poor health care was a "very big problem" for the country in the 2016 survey
hcproblem2015 = [51, 81, 57] #Percentage of respondents indicating that poor health care was a "very big problem" for the country in the 2015 survey
hcproblemchange = [35, 12, 25] #Change in percentage of respondents indicating that poor health care was a "very big problem" for the country between 2015 and 2016

##Questions on how big a priority improving health care should be
hcpriority2016 = [20, 14, 17] #Percentage of respondents indicating that improving health care should be the #1 priority of their country in the 2016 survey
hcpriority2015 = [33, 31, 16] #Percentage of respondents indicating that improving health care should be the #1 priority of their country in the 2015 survey
hcprioritychange = [-13, -17, 1] #Change in percentage of respondents indicating that improving health care should be the #1 priority of their country between the two surveys

##Questions on the economic situation
econproblem2016 = [6, 6, 10] #Percentage of respondents indicating that they thought their country's economic situation was "very good" in the 2016 survey
econproblem2015 = [11, 26, 20] #Percentage of respondents indicating that they thought their country's economic situation was "very good" in the 2015 survey 
econproblemchange = [-5, -20, -10] #Change between surveys, in percentage of respondents

##Questions on perceptions of the future of the economy
econoptimism2016 = [56, 86, 62] #Percentage of respondents who expected improvement in their country's economy within 12 months of the 2016 survey
econoptimism2015 = [53, 92, 45] #Percentage of respondents who expected improvement in their country's economy within 12 months of the 2015 survey
econoptimismchange = [3, -6, 17] #Change between surveys, in percentage of respondents

##Questions on economic stability
econinstability2016 = [77, 61, 61] #Percentage of respondents indicating they global economic instability a "major threat" to their country during the 2016 survey
econinstability2015 = [44, 48, 33] #Percentage of respondents indicating they were "very concerned" about global economic instability during the 2015 survey
econinstabilitychange = [33, 23, 28] #Change between surveys, in percentage of respondents

##Other (health) questions from 2016 survey
hcoptimism = [13, 47, 21] #Percentage of respondents indicating that, when children in their country grow up, "health care will be much better"
hcpersonal = [60, 59, 57] #Percentage of respondents indicating that there had been times in the past year where they had been unable to pay for necessary health/medical care for themselves or their family 

##Other (climate) questions from 2015 survey
climateimmediate = [54, 52, 31] #Percentage of respondents indicating that climate change is impacting people around the world now (as opposed to in the future)
climatepersonal = [61, 63, 39] #Percentage of respondents indicating that they were "very concerned" climate change would impact them personally during their lifetimes

##Pulling it all together
dict = {'Country':countries, 
        'Concern Over Health Care in 2016':hcproblem2016, 
        'Concern Over Health Care in 2015':hcproblem2015, 
        'Change in Concern Over Health Care':hcproblemchange, 
        'View of Health Care as a Major Priority in 2016':hcpriority2016,
        'View of Health Care as a Major Priority in 2015':hcpriority2015,
        'Change in View of Health Care as a Major Priority':hcprioritychange,
        'Concern Over Economy in 2016':econproblem2016,
        'Concern Over Economy in 2015':econproblem2015,
        'Change in Concern over Economy':econproblemchange,
        'Optimism About Economy in 2016':econoptimism2016,
        'Optimism About Economy in 2015':econoptimism2015,
        'Change in Optimism About Economy':econoptimismchange,
        'Fear of Economic Instability in 2016':econinstability2016,
        'Fear of Economic Instability in 2015':econinstability2015,
        'Change in Fear of Economic Instability':econinstabilitychange,
        'Optimism of Future Improvements in Health Care - 2016':hcoptimism,
        'Personal Experience With Inability to Pay for Health Care - 2016':hcpersonal,
        'Belief that Climate Change is an Immediate Problem - 2015':climateimmediate,
        'Belief that Climate Change Will Impact Respondent Personally - 2015':climatepersonal
       }
pewdata = pd.DataFrame(dict) #Creates a DataFrame compiling all of the data from Pew
pewdata.index = countries #Labels the DataFrame with the names of the countries, rather than 0-2
print(pewdata) #Confirmation of successful creation

In [ ]:
##Examining Correlations
##Matplotlib for charts (generally speaking, survey result 1 vs survey result 2, with countries being the datapoints for n = 3 samples)
##Look into if user input is an option for more efficient chart display?
##Ensure R value is printed out with each chart